# Techstart fouder matching

## Import library

In [6]:
import pandas as pd
import os
import datetime

c:\Users\nhson\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\nhson\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\nhson\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


## Constant

In [7]:
maxOfMemberPerTeam = 6
maxOfTeamPerUsers = 5
nameRoleUser = "Cá nhân mong muốn tìm được đội thi phù hợp"
nameRoleTeam = "Đội thi và đang tìm kiếm thêm thành viên"


In [12]:
namesCol={
    'Timestamp': 'time',
    'Bạn đăng ký tham gia ghép đội với vai trò là': 'role',
    'Họ và tên nhóm trưởng': 'nameLeader',
    'SĐT nhóm trưởng': 'sdtLeader',
    'Email nhóm trưởng': 'emailLeader',
    'Facebook nhóm trưởng': 'facebookLeader',
    'Lĩnh vực của dự án': 'typeProject',
    'Thông tin về cơ cấu nhân sự của dự án': 'infoMemberStruct',
    'Mô tả sơ lược ý tưởng của dự án': 'shortIdea',
    'Giai đoạn phát triển của dự án:': 'statusProject',
    'Số thành viên hiện có': 'numberMember',
    'Thông tin về kĩ năng của các thành viên trong đội': 'infoMemberSill',
    'Thông tin về thành tựu của các thành viên trong đội (nếu có)': 'infoGoalMember',
    'Số thành viên mong muốn kết nạp': 'numberRequestAddin',
    'Mô tả chân dung thành viên mong muốn kết nạp vào đội': 'profileRequest',
    'Thời gian dự kiến đồng hành': 'thoigiandonghanh',
    'Thế mạnh chuyên môn của thành viên mong muốn kết nạp (thành viên 1)': 'request01',
    'Thế mạnh chuyên môn của thành viên mong muốn kết nạp (thành viên 2)': 'request02',
    'Thế mạnh chuyên môn của thành viên mong muốn kết nạp (thành viên 3)': 'request03',
    'Thế mạnh chuyên môn của thành viên mong muốn kết nạp (thành viên 4)': 'request04',
    
    'Họ và tên của bạn': 'name_re',
    'Ngày tháng năm sinh của bạn': 'birth_re',
    'SĐT của bạn': 'sdt_re',
    'Email của bạn': 'email_re',
    'Facebook của bạn': 'facebook_re',
    '3 lĩnh vực bạn tự tin nhất ở bản thân': 'nicework_re',
    '3 thành tích ấn tượng nhất bạn đã đạt được ': 'profile_re',
    '3 lĩnh vực của đội thi mà bạn mong muốn được ghép cặp': 'teamRequest_re',
    'Bạn đã có ý tưởng rõ ràng chưa? Nếu có hãy mô tả sơ lược về ý tưởng mong muốn theo đổi là gì?': 'idea_re',
    }

dtypesCol = {
    "time": str,
    "role": str,
    "nameLeader": str,
    "sdtLeader": str,
    "emailLeader": str,
    "facebookLeader": str,
    "typeProject": str,
    "infoMemberStruct": str,
    "shortIdea": str,
    "statusProject": str,
    "numberMember": 'Int64',
    "numberRequestAddin": 'Int64',
    "profileRequest": str,
    "thoigiandonghanh": str,
    "request01": str,
    "request02": str,
    "request03": str,
    "request04": str,

    "name_re": str,
    "birth_re": str,
    "sdt_re": str,
    "email_re": str,
    "facebook_re": str,
    "nicework_re": str,
    "profile_re": str,
    "teamRequest_re": str,
    "idea_re": str,
    }

idLabel = ['Cơ khí & Chế tạo máy','Công nghệ sinh học','Công nghệ thông tin','Công nghệ môi trường','Công nghệ vật liệu','Công nghệ hóa học','Công nghệ điện - điện tử','Quản lý','Kinh doanh','Tài chính','Truyền thông']


## Data

### Read data

In [14]:
df = pd.read_csv('list.csv', header=0, names=list(namesCol.values()), dtype=dtypesCol)
df['id'] = range(0, len(df))

In [16]:
df = df.fillna(value={
    "thoigiandonghanh": "",
    "request01": "",
    "request02": "",
    "request03": "",
    "request04": "",

    "nicework_re": "",
    "teamRequest_re": "",
    "idea_re": "",
    })


In [17]:
df


,time,role,nameLeader,sdtLeader,emailLeader,facebookLeader,typeProject,infoMemberStruct,shortIdea,statusProject,...,name_re,birth_re,sdt_re,email_re,facebook_re,nicework_re,profile_re,teamRequest_re,idea_re,id
0,4/14/2023 18:33:22,Cá nhân mong muốn tìm được đội thi phù hợp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,qưdqw,2/22/0222,123123123,23123@lmao.com,kqndklqwemlwq.com,"Cơ khí & Chế tạo máy, Công nghệ thông tin, Kin...",ưqeqwe,"Cơ khí & Chế tạo máy, Kinh doanh, Truyền thông",ewqeqwe,0
1,4/15/2023 20:10:26,Đội thi và đang tìm kiếm thêm thành viên,Sơn,09434434123,daicaothunhson2003@gmail.com,fb.com/fdfdfá,"Cơ khí & Chế tạo máy, Công nghệ sinh học, Công...",Nguyễn Văn A - 1/1/2002 - Đại học Bách khoa H...,Vip pro,Dự án đang dừng lại tại mức có ý tưởng ban đầu,...,NaN,NaN,NaN,NaN,NaN,,NaN,,,1


### Pre process data

In [18]:
singleUsers = df.query("`role` == '{0}'".format(nameRoleUser)).loc[:,['id', 'time', 'nicework_re','teamRequest_re']]
teams = df.query("`role` == '{0}'".format(nameRoleTeam)).loc[:,['id', 'time', 'typeProject', 'numberMember', 'numberRequestAddin', 'request01','request02','request03','request04']]
print(len(singleUsers), "| ", len(teams))

1 |  1


In [19]:
def replanceLabel(_df, irow, col):
    arr = _df[irow][col].split(",")
    for j in range(len(arr)):
        ss = arr[j].strip()
        if ss in idLabel:
            arr[j] = idLabel.index(ss)
        else:
            arr[j] = -1
    _df[irow][col] = ','.join(map(str, arr))
    # _df[0, _df.columns.get_loc('id')] = ','.join(map(str, arr))
    # _df.at[irow, col] = ','.join(map(str, arr))
def replanceLabels(_df, cols):
    for i in range(len(_df)):
        for col in cols:
            replanceLabel(_df, i, col)
def fileWritelabels(tempf, line):
    labels = line.split(",")
    tempf.write("{0} ".format(len(labels)))
    for label in labels:
        tempf.write("{0} ".format(label))

In [20]:
line_singleUsers = singleUsers.values.tolist()
line_teams = teams.values.tolist()

In [21]:
replanceLabels(line_singleUsers, [2, 3])
replanceLabels(line_teams, [2,5,6,7,8])

In [22]:
for i in range(len(line_singleUsers)):
    line_singleUsers[i][1] = int(datetime.datetime.strptime(line_singleUsers[i][1], '%m/%d/%Y %H:%M:%S').timestamp())
    line_singleUsers[i].append(maxOfTeamPerUsers)

In [23]:
line_teams_std = []
for team in line_teams:
    arr = []
    arr.append(team[0])
    arr.append(int(datetime.datetime.strptime(team[1], '%m/%d/%Y %H:%M:%S').timestamp()))
    arr.append(team[2])
    request_temp = []
    for i in range(4, len(team)):
        if team[i] != '-1':
            request_temp.append(team[i])
    arr.append(min([maxOfMemberPerTeam - team[3], team[4], len(request_temp)]))
    arr.append(','.join(map(str, request_temp)))
    line_teams_std.append(arr)

## Create file output

In [24]:
with open('temp.inp', "wt") as tempf:
    # Users
    tempf.write("{0}\n".format(len(line_singleUsers)))
    for line in line_singleUsers:
        tempf.write("{0} {1} {2}\n".format(line[0], line[1], line[4]))
        fileWritelabels(tempf, line[2])
        tempf.write("\n")
        fileWritelabels(tempf, line[3])
        tempf.write("\n")
    
    # Teams
    tempf.write("{0}\n".format(len(line_teams_std)))
    for line in line_teams_std:
        tempf.write("{0} {1} {2}\n".format(line[0], line[1], line[3]))
        fileWritelabels(tempf, line[2])
        tempf.write("\n")
        fileWritelabels(tempf, line[4])
        tempf.write("\n")
    
    # Del node
    tempf.write("{0}\n".format(0))

    # Del edge
    tempf.write("{0}\n".format(1))
    tempf.write("{0} {1}\n".format(2, 0))

Using c++ to process data for super fast tay to vl

In [25]:
os.system("hotaMatching.exe")

0

## Read output

In [26]:
with open("temp.out", "rt") as tempf:
    result = tempf.read()
result = result.strip().split("\n")
for i in range(len(result)):
    result[i] = result[i].strip().split(" ")

In [27]:
result

[['1', '0']]